# Exploring Star Catalogs in VR

Download real astronomical data and explore stars in 3D space.

In [ ]:
# Install required packages
# !pip install astropy astroquery pandas numpy immersivepoints

In [ ]:
# Auto-install in current kernel
import sys
import subprocess
import importlib

try:
    import immersivepoints as ip
except ImportError:
    print("Installing immersivepoints...")
    subprocess.check_call([sys.executable, '-m', 'pip', 'install', 'immersivepoints', '--quiet'])
    import site
    importlib.reload(site)
    import immersivepoints as ip

import numpy as np
import pandas as pd
from astropy import units as u
from astropy.coordinates import SkyCoord

print("✓ Astronomy VR Visualization Ready!")

## Option 1: Generate Synthetic Nearby Stars

For quick testing, we can generate synthetic star data based on real star density.

In [ ]:
def generate_nearby_stars(n_stars=5000, max_distance=100):
    """
    Generate synthetic star field based on realistic distribution.
    Distance in light-years.
    """
    # Random positions in spherical coordinates
    distances = np.random.uniform(1, max_distance, n_stars) ** (1/3) * max_distance ** (2/3)
    theta = np.random.uniform(0, 2*np.pi, n_stars)  # Azimuthal angle
    phi = np.arccos(np.random.uniform(-1, 1, n_stars))  # Polar angle
    
    # Convert to Cartesian
    x = distances * np.sin(phi) * np.cos(theta)
    y = distances * np.sin(phi) * np.sin(theta)
    z = distances * np.cos(phi)
    
    # Generate B-V color (temperature indicator)
    # Most stars are K and M type (red), fewer O,B,A (blue)
    bv_colors = np.random.beta(2, 5, n_stars) * 2.0  # Range 0-2
    
    # Convert B-V to hue (blue to red)
    # B-V: -0.3 (blue) to 2.0 (red)
    hue = np.clip((bv_colors + 0.3) / 2.3, 0, 1)
    # Invert so blue stars have blue color
    hue = 1.0 - hue  # Now 1.0 = blue, 0.0 = red
    hue = hue * 0.7  # Map to hue range: 0.7 (blue) to 0.0 (red)
    
    return np.column_stack([x, y, z, hue]).astype(np.float32)

print("Generating 5000 synthetic nearby stars...")
points = generate_nearby_stars(n_stars=5000, max_distance=100)

# Scale for VR (1 light-year = 0.1 VR units)
points[:, :3] *= 0.1

print(f"Generated {len(points)} stars")
print(f"Distance range: {points[:,:3].min():.1f} to {points[:,:3].max():.1f} VR units")
print(f"(Represents {points[:,:3].max()/0.1:.0f} light-years)")

## Option 2: Use Real Gaia Data (Requires Internet)

Uncomment and run this cell to download real star data from the Gaia mission.

In [ ]:
# # Download nearest stars from Gaia DR3
# from astroquery.gaia import Gaia
# 
# print("Querying Gaia DR3 for nearest stars...")
# print("This may take 30-60 seconds...")
# 
# # Query nearest 10,000 stars with good parallax measurements
# query = """
# SELECT TOP 10000
#     ra, dec, parallax, phot_g_mean_mag, bp_rp
# FROM gaiadr3.gaia_source
# WHERE parallax > 10 AND parallax_over_error > 5
# ORDER BY parallax DESC
# """
# 
# job = Gaia.launch_job(query)
# results = job.get_results()
# 
# # Convert to pandas for easier handling
# df = results.to_pandas()
# 
# # Convert parallax (mas) to distance (light-years)
# # distance (pc) = 1000 / parallax(mas)
# # distance (ly) = distance(pc) * 3.26156
# df['distance_ly'] = (1000.0 / df['parallax']) * 3.26156
# 
# # Convert RA, Dec, Distance to Cartesian coordinates
# ra_rad = np.radians(df['ra'].values)
# dec_rad = np.radians(df['dec'].values)
# dist = df['distance_ly'].values
# 
# x = dist * np.cos(dec_rad) * np.cos(ra_rad)
# y = dist * np.cos(dec_rad) * np.sin(ra_rad)
# z = dist * np.sin(dec_rad)
# 
# # Color by BP-RP (blue-red color)
# bp_rp = df['bp_rp'].fillna(1.0).values
# hue = np.clip(1.0 - (bp_rp + 0.5) / 3.0, 0, 1)  # Blue=1, Red=0
# hue = hue * 0.7  # Map to 0-0.7 range
# 
# points = np.column_stack([x, y, z, hue]).astype(np.float32)
# points[:, :3] *= 0.1  # Scale for VR
# 
# print(f"Downloaded {len(points)} real stars from Gaia!")

## Add Famous Stars with Labels

Let's mark some famous nearby stars.

In [ ]:
# Famous nearby stars (approximate positions in light-years)
famous_stars = {
    'Alpha Centauri': ([4.37, 0, 0], 0.15),  # Yellow, G-type
    'Sirius': ([8.6, 2, -1], 0.5),  # White, A-type
    'Vega': ([25, 0, 19], 0.6),  # Blue-white, A-type
    'Arcturus': ([37, 0, 0], 0.05),  # Orange, K-type  
    'Betelgeuse': ([548, 100, -200], 0.0),  # Red supergiant
}

# Add these stars to our point cloud (scaled for VR)
famous_points = []
for name, (pos, hue) in famous_stars.items():
    scaled_pos = [p * 0.1 for p in pos]
    famous_points.append([*scaled_pos, hue])

famous_array = np.array(famous_points, dtype=np.float32)

print("Added famous stars:")
for name, (pos, _) in famous_stars.items():
    print(f"  - {name}: {pos} light-years from Sun")

## Visualize in Jupyter Notebook

In [ ]:
# Combine regular stars and famous stars
all_stars = np.vstack([points, famous_array])

# Render inline
ip.renderPoints(all_stars, point_size=0.05, background_color=0x000000, show_axes=True)

## Generate VR Link

In [ ]:
ip.showVR(all_stars, point_size=0.05)

## Create Star Cluster (Pleiades-like)

Generate a tight cluster of young, hot stars.

In [ ]:
def create_star_cluster(n_stars=500, cluster_radius=10, distance=400):
    """
    Create a spherical star cluster.
    """
    # Random positions in sphere
    theta = np.random.uniform(0, 2*np.pi, n_stars)
    phi = np.arccos(np.random.uniform(-1, 1, n_stars))
    r = np.random.uniform(0, cluster_radius, n_stars) ** (1/3) * cluster_radius ** (2/3)
    
    # Cluster in Cartesian coords (centered at distance)
    x = r * np.sin(phi) * np.cos(theta) + distance
    y = r * np.sin(phi) * np.sin(theta)
    z = r * np.cos(phi)
    
    # Young clusters have hot, blue stars
    hue = np.random.uniform(0.5, 0.7, n_stars)  # Blue-white range
    
    cluster = np.column_stack([x, y, z, hue]).astype(np.float32)
    cluster[:, :3] *= 0.1  # Scale for VR
    
    return cluster

print("Creating Pleiades-like star cluster...")
cluster = create_star_cluster(n_stars=500, cluster_radius=8, distance=444)

print(f"Cluster created at ~444 light-years distance")
print(f"Contains {len(cluster)} young, hot stars")

# Visualize cluster alone
ip.renderPoints(cluster, point_size=0.08, background_color=0x000000)

## Save Your Star Map

In [ ]:
# Save the star field
output_file = "nearby_stars_100ly.xyzi"
all_stars.astype(np.float32).byteswap().tofile(output_file)

print(f"Saved {len(all_stars)} stars to {output_file}")
print(f"File size: {len(all_stars) * 16 / 1024:.1f} KB")
print(f"\nUpload at: https://immersivepoints.com/upload.html")

## Next Steps

**In VR, you can:**
- Walk from our Sun to nearby stars
- See why constellations look different from other stars' perspectives
- Observe the 3D structure of star clusters
- Understand stellar density gradients

**Try modifying:**
- Increase `max_distance` for a larger star field
- Change color mapping (by magnitude instead of temperature)
- Add multiple clusters at different distances
- Query real data from Gaia (uncomment Option 2 above)